Find possible instances of scriptural and apocryphal citations 

In [1]:
import sys 
sys.path.append('../')
from lib.standardization import * 

In [ ]:
def replaceBook(note): 
    note = note.split(" ")
    for idx, word in enumerate(note): 
        word = re.sub("v","u",word)
        word = re.sub(r"^i","j",word)
        if word in bible_dict:
            note[idx] = bible_dict[word]
    return " ".join(note) 

'''Converts numbered books into a single string, e.g., '1 corinthians' to '1corinthians' for ease of processing'''
def replaceNumBook(text):
    book = " ".join(text.split(" ")[0:2])
    if book in numBook:  
        text = re.sub(rf'{book}', numBook[book], text)
    return text

In [ ]:
import csv, re

# outfile = open("sermons_possible_citations.csv","w+")
# writer = csv.DictWriter(outfile, fieldnames=["tcp_id","div_path","sourceline","sourcepos","note"])
# writer.writeheader()

with open('sermons_marginalia.csv', 'r') as file:          # Read lines separately
    notes = csv.reader(file, delimiter=',')
    # find instances of possible numbered books
    known, unknown = {}, {}  
    for idx, entry in enumerate(notes):
        # get note text 
        n = entry[-1]
        # replace all periods with spaces and convert to lower case 
        n = re.sub(r'(\.)',r' ',n).lower()
        # replace all instances of two or more spaces with a single space. 
        n = re.sub(r'\s+',' ',n)
        possible = re.findall(r'\b([a-z]+)\b \b[0-9•]+\b \b[0-9•v]+\b', n)